In [51]:
import pandas as pd
from pandas import Series
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
data = pd.read_csv('/content/Fraud_check.csv')

In [4]:
data.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [5]:
from sklearn.preprocessing import LabelEncoder

In [7]:
encoder = LabelEncoder()
for i in data.columns:
    print(i)
    if data[i].dtypes == object:
        data[i] = encoder.fit_transform(data[i])
    else:
        pass

Undergrad
Marital.Status
Taxable.Income
City.Population
Work.Experience
Urban


In [9]:
data.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,0,2,68833,50047,10,1
1,1,0,33700,134075,18,1
2,0,1,36925,160205,30,1
3,1,2,50190,193264,15,1
4,0,1,81002,27533,28,0


In [15]:
data.rename(columns= {'Marital.Status': 'mar_st', 'Taxable.Income': 'tax_inc', 'City.Population': 'city_pl', "Work.Experience": 'work_exp' }, inplace=True)

In [16]:
data['Status'] = data['tax_inc'].apply(lambda tax_inc: 'Risky' if tax_inc <= 30000 else 'good')

In [17]:
data.drop(['tax_inc'], axis =1, inplace=True)

In [19]:
data

,Undergrad,mar_st,city_pl,work_exp,Urban,Status
0,0,2,50047,10,1,good
1,1,0,134075,18,1,good
2,0,1,160205,30,1,good
3,1,2,193264,15,1,good
4,0,1,27533,28,0,good
...,...,...,...,...,...,...
595,1,0,39492,7,1,good
596,1,0,55369,2,1,good
597,0,0,154058,0,1,good
598,1,1,180083,17,0,good


In [20]:
data['Status'] = encoder.fit_transform(data['Status'])

In [21]:
data

,Undergrad,mar_st,city_pl,work_exp,Urban,Status
0,0,2,50047,10,1,1
1,1,0,134075,18,1,1
2,0,1,160205,30,1,1
3,1,2,193264,15,1,1
4,0,1,27533,28,0,1
...,...,...,...,...,...,...
595,1,0,39492,7,1,1
596,1,0,55369,2,1,1
597,0,0,154058,0,1,1
598,1,1,180083,17,0,1


In [29]:
array = data.values
X = array[:, 0:5]
Y = array[:, -1]
seed = 7

In [30]:
y.value_counts()

1    476
0    124
Name: Status, dtype: int64

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

#Bagging Method

In [63]:
#Bagging Decision Tree for Classification

kfold = KFold(n_splits=8, random_state=seed)
cart = DecisionTreeClassifier()
num_trees= 100
model = BaggingClassifier(base_estimator= cart, n_estimators= num_trees, random_state=seed)
model.fit(X_train, Y_train)
results = cross_val_score(model, X, Y, cv = kfold)
print(results.mean())

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.7316666666666667


In [64]:
#Predicting on the test data
pred = model.predict(X_test)
pd.Series(pred).value_counts()

1    165
0     15
dtype: int64

In [65]:
pd.crosstab(Y_test, pred)

col_0,0,1
row_0,,
0,3,37
1,12,128


In [66]:
print("Accuracy: ", np.mean(pred==Y_test))

Accuracy:  0.7277777777777777


Bagging method accuracy on the test data is 72.77%

#Random Forest

In [49]:
#Random Forest

num_trees = 100
max_features = 3
kfold = KFold(n_splits=8, random_state=seed)
model1 = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model1.fit(X_train, Y_train)
result2 = cross_val_score(model1, X, Y, cv=kfold)
print(result2.mean())

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.7383333333333333


In [53]:
#Predicting on the rest data
preds = model1.predict(X_test)
pd.Series(preds).value_counts()

1    168
0     12
dtype: int64

In [54]:
pd.crosstab(Y_test, preds)

col_0,0,1
row_0,,
0,1,39
1,11,129


In [58]:
print("Accuracy: ", np.mean(preds==Y_test))

Accuracy:  0.7222222222222222


Random Forest Accuracy on test data is 72%

#AdaBoost

In [59]:
#Adaboost

num_trees = 20
kfold= KFold(n_splits=8, random_state=seed)
model2 = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
model2.fit(X_train, Y_train)
result2 = cross_val_score(model2, X, Y, cv=kfold)
print(result2.mean())

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.7916666666666667


In [60]:
#Predicting on the rest data
preds1 = model2.predict(X_test)
pd.Series(preds1).value_counts()

1    178
0      2
dtype: int64

In [61]:
pd.crosstab(Y_test, preds1)

col_0,0,1
row_0,,
0,0,40
1,2,138


In [62]:
print("Accuracy: ", np.mean(preds1==Y_test))

Accuracy:  0.7666666666666667


AdaBoost accuracy on test data is 76.66%